In [102]:
import os
from datetime import datetime
import shutil
import yaml
from tqdm.notebook import tqdm as tqdm
import torch
import numpy as np
import sys
%matplotlib widget
import matplotlib as mpl
from matplotlib import pyplot as plt 


import scipy.io as sio
import logging
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from collections import namedtuple, defaultdict
import transformers as tr
from transformers import pipeline
import pandas as pd 
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise 
from itertools import cycle, islice
import scipy as sc
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import bcolz
import pickle

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


In [3]:
save_path="/Users/eghbalhosseini/MyData/semantic_proj/"
analysis_path="/Users/eghbalhosseini/MyData/semantic_proj/analysis/"
analysis_type='gp2_vs_glove'
model_type="gpt2-xl"
LAYER_COUNT = 48
FEATURE_COUNT = 1600
access_rights = 0o755
try:
    os.mkdir(save_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_path)
model_path=os.path.join(save_path,model_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)
    
try:
    os.mkdir(analysis_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%analysis_path)
model_path=os.path.join(analysis_path,analysis_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)

Creation of /Users/eghbalhosseini/MyData/semantic_proj/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/gpt2-xl failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/analysis/ failed



In [4]:
# configure the gpt2 model 
config=GPT2Config.from_pretrained(model_type)
config.output_hidden_states=True
# make model from config
model=GPT2Model(config)
model.from_pretrained(model_type)
device = torch.device("cpu")
model.to(device)
# 
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

In [5]:
def match_tokenized_to_untokenized_gpt2(tokenized_sent, untokenized_sent):
    mapping = defaultdict(list)
    untokenized_sent_index = 0
    tokenized_sent_index = 0
    while (untokenized_sent_index < len(untokenized_sent) and
        tokenized_sent_index < len(tokenized_sent)):
      while (tokenized_sent_index + 1 < len(tokenized_sent) and
          not(tokenized_sent[tokenized_sent_index + 1].startswith("Ġ"))):
        mapping[untokenized_sent_index].append(tokenized_sent_index)
        tokenized_sent_index += 1
      mapping[untokenized_sent_index].append(tokenized_sent_index)
      untokenized_sent_index += 1
      tokenized_sent_index += 1
    return mapping

# extract words from semtantic projection paper 

In [9]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')
Domains = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Domains top 25',skiprows=1)


In [10]:
lineList=Domains['Animals']


In [11]:
representation_arr = np.zeros((len(lineList),2), dtype=np.object)

In [12]:
# get sentence lines 
for index, line in tqdm(enumerate(lineList)):
    untokenized_word = line.strip().split()
    tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
    untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
    segment_ids = [1 for x in tokenized_word]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
        last_hidden_state=encoded_layers[0]
        hidden_output=encoded_layers[2]
        layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
        layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
        rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
        representation_arr[index,0]=' '.join(untokenized_word)
        representation_arr[index,1]=rep_arr

25it [00:11,  2.09it/s]


In [ ]:
plt.figure(figsize=[25,25])
for i, x in enumerate(representation_arr):
    plt.subplot(5,5,i+1)
    NN_response=np.stack(x[1]);
    sort_ind=np.argsort(NN_response[-2,:])
    plt.imshow(NN_response[:,sort_ind],aspect=25)
    plt.title(x[0])

In [ ]:
plt.figure(figsize=[25,25]);
i=5
NN_response_1=np.stack(representation_arr[i][1]);
for k, x in enumerate(representation_arr):
    plt.subplot(5,5,k+1)
    NN_response_2=np.stack(x[1]);
    plt.imshow(pairwise.euclidean_distances(NN_response_1,NN_response_2))
    plt.title(representation_arr[i][0]+' vs. '+representation_arr[k][0])

# extract word embedding for the descriptors 

In [15]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')

In [16]:
lineList=Dimensions.iloc[0][1:]

In [17]:
dimensions_arr = np.zeros((len(lineList),2), dtype=np.object)

In [18]:
# get sentence lines 
for index, line in tqdm(enumerate(lineList)):
    untokenized_word = line.strip().split()
    tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
    untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
    segment_ids = [1 for x in tokenized_word]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
        last_hidden_state=encoded_layers[0]
        hidden_output=encoded_layers[2]
        layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
        layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
        rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
        dimensions_arr[index,0]=' '.join(untokenized_word)
        dimensions_arr[index,1]=rep_arr

6it [00:02,  2.59it/s]


In [ ]:
plt.figure(figsize=[25,25])
for i, x in enumerate(dimensions_arr):
    plt.subplot(3,3,i+1)
    NN_response=np.stack(x[1]);
    plt.imshow(NN_response,aspect=25)
    plt.title(x[0])

In [ ]:
plt.figure(figsize=[25,25]);
i=1
NN_response_1=np.stack(dimensions_arr[i][1]);
for k, x in enumerate(dimensions_arr):
    plt.subplot(5,5,k+1)
    NN_response_2=np.stack(x[1]);
    plt.imshow(pairwise.cosine_similarity(NN_response_1,NN_response_2))
    plt.title(dimensions_arr[i][0]+' vs. '+dimensions_arr[k][0])

# use projection to align the words along a specific dimension


In [21]:
dim_positive=dimensions_arr[0:3]
dim_negative=dimensions_arr[3:]

In [22]:
sem_vec=[[np.stack(y[1])-np.stack(x[1]) for y in dim_negative] for x in dim_positive]

In [23]:
sem_vec_1=[np.stack(x) for x in sem_vec]

In [24]:
sem_vec_2=np.concatenate(sem_vec_1,axis=0)

In [25]:
sem_vec_ave=np.mean(sem_vec_2,axis=0)

In [26]:
sem_vec_ave.shape

(49, 1600)

In [27]:
X = np.arange(0, 49)
Y=X
X, Y = np.meshgrid(X, Y)
plt.imshow(pairwise.cosine_similarity(sem_vec_ave,sem_vec_ave))

In [28]:
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, pairwise.cosine_similarity(sem_vec_ave,sem_vec_ave), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# get the word projection onto the dimension axis across layers 


(49, 1600)

In [30]:
NN_resp_projection=[]
NN_resp_names=[]
for i, x in enumerate(representation_arr):
    NN_response=np.stack(x[1]);
    a=np.dot(sem_vec_ave,NN_response.transpose())
    NN_resp_projection.append(np.diag(a))
    NN_resp_names.append(x[0])

In [ ]:
fig = plt.figure(figsize=[10,10])
[plt.plot(np.arange(0,a.shape[0]),a,label=NN_resp_names[i]) for i,a in enumerate(NN_resp_projection)]
plt.legend(loc='upper left')

In [32]:
#fig = plt.figure(figsize=[10,10])
#plt.axis([0, 10, 0, 10])
#[[plt.text(k,m,NN_resp_names[i]) for k,m in enumerate(a)] for i,a in enumerate(NN_resp_projection)]
#for i, x in enumerate(NN_resp_projection): 
    
#    plt.text()
#NN_resp_projection[1]
tab10 = cm.get_cmap('tab10', len(NN_resp_projection))
tab10(2)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [ ]:
fig = plt.figure(figsize=[25,20])
ax=plt.subplot(1,1,1)
plt.plot()
for k, m in enumerate(NN_resp_projection):
    plt.plot(m,'.',color=tab10(k),markersize=1)
    #plt.plot(np.asarray([k,k]),np.asarray([np.min(m),np.max(m)]))
    #plt.show() 
    for i,a in enumerate(m):
        ax.text(i,a,NN_resp_names[k],{'color': tab10(k), 'fontsize': 6},ha='left',va='center')
ylims=plt.ylim()
[plt.plot(np.asarray([i,i]),ylims,color='k',linewidth=.5) for i,_ in enumerate(NN_resp_projection[1])]

# compare embedding from GPT2 against GloVe

## run glove extraction of it is not ran before 

In [50]:
words = []
idx = 0
word2idx = {}
glove_path='/Users/eghbalhosseini/MyData/semantic_proj/GLoVe/'
vectors = bcolz.carray(np.zeros(0), rootdir=f'{glove_path}/42B.300.dat', mode='w')

with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
    for l in tqdm(f):
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

1917494it [07:04, 4518.81it/s]


In [36]:
vectors = bcolz.carray(vectors[0:].reshape((-1, 300)), rootdir=f'{glove_path}/42B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'{glove_path}/42B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'{glove_path}/42B.300_idx.pkl', 'wb'))

carray((575248201,), float64)
  nbytes := 4.29 GB; cbytes := 4.09 GB; ratio: 1.05
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 2048; chunksize: 16384; blocksize: 16384
  rootdir := '/Users/eghbalhosseini/MyData/semantic_proj/GLoVe//42B.300.dat'
  mode    := 'w'
[ 0.        0.18378  -0.12123  ..., -0.090892  0.27827   0.11374 ]

## otherwise load glove

In [58]:
vectors = bcolz.open(f'{glove_path}/42B.300.dat')[:]
words = pickle.load(open(f'{glove_path}/42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/42B.300_idx.pkl', 'rb'))


In [59]:
glove = {w: vectors[word2idx[w]] for w in words}

In [61]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')
lineList=Dimensions.iloc[0][1:]

In [63]:
dimensions_arr=[glove[x] for x in lineList]

In [87]:
dim_positive=dimensions_arr[0:3]
dim_negative=dimensions_arr[3:]

sem_vec=[[np.stack(y)-np.stack(x) for y in dim_negative] for x in dim_positive]
sem_vec_1=[np.stack(x) for x in sem_vec]
sem_vec_2=np.concatenate(sem_vec_1,axis=0)
glove_sem_vec_ave=np.mean(sem_vec_2,axis=0)

In [75]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')
Domains = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Domains top 25',skiprows=1)
# 
lineList=Domains['Animals']
Glove_rep_arr = np.zeros((len(lineList),2), dtype=np.object)

In [81]:
# get sentence lines 
for index, line in tqdm(enumerate(lineList)):
   
    Glove_rep_arr[index,0]=line
    Glove_rep_arr[index,1]=glove[line]

25it [00:00, 140371.62it/s]


In [90]:
Glove_resp_projection=[]
Glove_resp_names=[]
for i, x in enumerate(Glove_rep_arr):
    Glove_response=x[1]
    a=np.dot(glove_sem_vec_ave,Glove_response.transpose())
    Glove_resp_projection.append(a)
    Glove_resp_names.append(x[0])

In [130]:
NN_rep_proj_mat=np.stack(NN_resp_projection).transpose()


SyntaxError: unexpected EOF while parsing (<ipython-input-130-146e5d885eca>, line 2)

In [170]:
plt.figure(figsize=[20,25])
x=np.asarray(Glove_resp_projection).reshape(-1, 1)
for i, y in enumerate(NN_rep_proj_mat):
    regr = linear_model.LinearRegression()
    y=y.reshape(-1, 1)
    regr.fit(x, y)
    y_pred = regr.predict(x)
    plt.subplot(10,5,i+1)
    plt.scatter(x,y,s=5,c=[0,0,0])
    plt.plot(x, y_pred, color='blue', linewidth=3)
    a=r2_score(y,y_pred)
    s=f"{a:.3f}"
    plt.title('R squared: '+s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [165]:
a=r2_score(y,y_pred)

In [168]:
s=f"{a:.3f}"

In [169]:
s

'0.043'

# test a sentence 

1it [00:00,  2.77it/s]


In [46]:
plt.figure()
plt.imshow(dimensions_arr[0][1][47],aspect='auto')
plt.show()
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [103]:
Data_path='/Users/eghbalhosseini/MyData/ecog_DNN/sentence_sampling/'
s_dat=pd.read_pickle(os.path.join(Data_path,'sentence_data_outlier_detect.pkl'))
s_with_all_feat_out=[]
feat_names=['prevalence',
             'concreteness',
             'word length',
             'lexical freq',
             'arousal',
             'valence',
             'surprisal',
             'sent dependency length',
            'sent_fwprob5',
            'sent_unigram_surp',
            'sent_totalsurp']
[s_with_all_feat_out.append(s_dat[x]) for x in feat_names]

model_state_out=[]
model_names=['bert_state','gpt2_state','tiny_lstm_state','ordered_neurons_state']
[model_state_out.append(s_dat[x]) for x in model_names]
s_string_with_all_feat_out=s_dat['sentence']

In [105]:
sent=s_string_with_all_feat_out
dimensions_arr = np.zeros((len(sent),2), dtype=np.object)


In [106]:
# get sentence lines 
for index, line in tqdm(enumerate(sent)):
    untokenized_word = line.strip().split()
    tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
    untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
    segment_ids = [1 for x in tokenized_word]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
        last_hidden_state=encoded_layers[0]
        hidden_output=encoded_layers[2]
        layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
        layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
        rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
        dimensions_arr[index,0]=' '.join(untokenized_word)
        dimensions_arr[index,1]=rep_arr

In [111]:
np.save('/Users/eghbalhosseini/MyData/ecog_DNN/sentence_sampling/gpt2_states_from_huggingface.npy', dimensions_arr, allow_pickle=True, fix_imports=True)


In [108]:
len(dimensions_arr)

4629

In [95]:
[np.max(x) for x in dimensions_arr[0][1]]

[0.1359662,
 4.352357,
 5.6076064,
 6.745062,
 8.518669,
 9.29471,
 9.802319,
 10.263653,
 10.432583,
 11.664098,
 12.359014,
 13.67273,
 14.751403,
 17.016434,
 17.362995,
 17.758003,
 21.52797,
 22.674604,
 22.965948,
 22.942162,
 23.33326,
 24.13773,
 23.26604,
 22.043873,
 20.925617,
 21.43827,
 21.20632,
 21.059341,
 21.572973,
 20.881594,
 22.675613,
 22.949026,
 23.027697,
 22.701693,
 22.43643,
 22.220613,
 22.405735,
 23.844685,
 26.139372,
 26.659834,
 25.98666,
 27.897467,
 26.242613,
 26.889782,
 28.600185,
 29.764103,
 29.084406,
 30.561567,
 3.929636]

In [70]:
np.matmul(gpt2_state_data[1],gpt2_state_data[1])

(13, 1600)

In [82]:
plt.figure()
for x in dimensions_arr[0][1]: 
    plt.plot(np.matmul(x,gpt2_state_data[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…